In [5]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.7 MB/s eta 0:00:00


In [7]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [8]:
model = T5ForConditionalGeneration.from_pretrained('t5-large')
tokenizer = T5Tokenizer.from_pretrained('t5-large', model_max_length=1024)

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [8]:
text = """I love my cat because """
inputs = tokenizer(["summarize: " + text], return_tensors="pt")

In [9]:
summary = model.generate(**inputs, return_dict_in_generate=True, max_length=30, output_scores=True)

<class 'transformers.generation.utils.GreedySearchEncoderDecoderOutput'>
<pad>i love my cat because he is so sweet. he is a good listener and he is a good listen


In [10]:
"len(summary.sequences[0]): ", len(summary.sequences[0])

('len(summary.sequences[0]): ', 30)

In [11]:
print("len(summary.scores): ", len(summary.scores))
print("summary.scores[1].shape: ", summary.scores[1].shape)

len(summary.scores):  29
summary.scores[1].shape:  torch.Size([1, 32128])


In [12]:
!git clone https://github.com/jwkirchenbauer/lm-watermarking.git

Cloning into 'lm-watermarking'...
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 313 (delta 10), reused 5 (delta 5), pack-reused 288
Receiving objects: 100% (313/313), 11.98 MiB | 16.02 MiB/s, done.
Resolving deltas: 100% (87/87), done.


In [13]:
import sys
sys.path.insert(0, "/content/lm-watermarking")

In [14]:
from watermark_processor import WatermarkDetector, WatermarkLogitsProcessor
from transformers import (LogitsProcessorList)

In [15]:
 input_text = (
        "The diamondback terrapin or simply terrapin (Malaclemys terrapin) is a "
        "species of turtle native to the brackish coastal tidal marshes of the "
        "Northeastern and southern United States, and in Bermuda.[6] It belongs "
        "to the monotypic genus Malaclemys. It has one of the largest ranges of "
        "all turtles in North America, stretching as far south as the Florida Keys "
        "and as far north as Cape Cod.[7] The name 'terrapin' is derived from the "
        "Algonquian word torope.[8] It applies to Malaclemys terrapin in both "
        "British English and American English. The name originally was used by "
        "early European settlers in North America to describe these brackish-water "
        "turtles that inhabited neither freshwater habitats nor the sea. It retains "
        "this primary meaning in American English.[8] In British English, however, "
        "other semi-aquatic turtle species, such as the red-eared slider, might "
        "also be called terrapins. The common name refers to the diamond pattern "
        "on top of its shell (carapace), but the overall pattern and coloration "
        "vary greatly. The shell is usually wider at the back than in the front, "
        "and from above it appears wedge-shaped. The shell coloring can vary "
        "from brown to grey, and its body color can be grey, brown, yellow, "
        "or white. All have a unique pattern of wiggly, black markings or spots "
        "on their body and head. The diamondback terrapin has large webbed "
        "feet.[9] The species is"
)

# output_text = "watermarking language models can mitigate potential harms. watermarks can be embedded with"

In [16]:
watermark_processor = WatermarkLogitsProcessor(vocab=list(tokenizer.get_vocab().values()),
                                               gamma=0.25,
                                               delta=2.0,
                                               seeding_scheme="simple_1")

tokenized_input = tokenizer(input_text, return_tensors="pt").to(model.device)
print(tokenized_input["input_ids"].shape)
# note that if the model is on cuda, then the input is on cuda
# and thus the watermarking rng is cuda-based.
# This is a different generator than the cpu-based rng in pytorch!

output_tokens = model.generate(**tokenized_input,
                               logits_processor=LogitsProcessorList([watermark_processor]), max_new_tokens=50, return_dict_in_generate=True, max_length=30, output_scores=True)

# if decoder only model, then we need to isolate the
# newly generated tokens as only those are watermarked, the input/prompt is not
# output_tokens = output_tokens[:,tokenized_input["input_ids"].shape[-1]:]
output_text = tokenizer.batch_decode(output_tokens.sequences, skip_special_tokens=True)[0]
output_text

torch.Size([1, 356])


Both `max_new_tokens` (=50) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'on the back of their shells, and also has large, webbed endemic to Bermuda, the Bahamas, and the Bahamas.[7] found in the Atlantic Ocean and the Pacific Ocean. found in the Atlantic Ocean'

In [17]:
output_text = "hello everybody my name is pouya"

In [18]:
watermark_detector = WatermarkDetector(vocab=list(tokenizer.get_vocab().values()),
                                        gamma=0.25, # should match original setting
                                        seeding_scheme="simple_1", # should match original setting
                                        device="cpu", # must match the original rng device type
                                        tokenizer=tokenizer,
                                        z_threshold=4.0,
                                        normalizers=[],
                                        ignore_repeated_bigrams=False)

score_dict = watermark_detector.detect(output_text) # or any other text of interest to analyze

In [19]:
score_dict

{'num_tokens_scored': 7,
 'num_green_tokens': 3,
 'green_fraction': 0.42857142857142855,
 'z_score': 1.091089451179962,
 'p_value': 0.13761676203741713,
 'prediction': False}

## CNN dataset

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

In [3]:
ds = load_dataset('cnn_dailymail', '3.0.0')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [21]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-large')

articles = [d['article'] for d in ds['train']]
summaries = [d['highlights'] for d in ds['train']]

# inputs = tokenizer(articles, truncation=True, padding='longest', return_tensors='pt')
# targets = tokenizer(summaries, truncation=True, padding='longest', return_tensors='pt')

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [22]:
type(articles)

list

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.4 MB/s eta 0:00:00
  Created wheel for lmppl: filename=lmppl-0.3.1-py3-none-any.whl size=13113 sha256=247bfd49f250733dcb88e892cce3c308c9c3c9326f5f398a2d08bc4fdf9fd3bc
  Stored in directory: /root/.cache/pip/wheels/c2/19/b5/3ff20891be9a8cb50595b6d4d37b819be0cbe71ddcea9ac10c
Successfully built lmppl
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=

In [10]:
import lmppl

In [11]:
scorer = lmppl.EncoderDecoderLM('google/flan-t5-large')

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


In [14]:
inputs = [
    'I dropped my laptop on my knee, and someone stole my coffee.',
    'I dropped my laptop on my knee, and someone stole my coffee.'
]
outputs = [
    'and the door is a blackboard.',
    'I am sad.'
]
ppl = scorer.get_perplexity(input_texts=inputs, output_texts=outputs)
ppl
# print(list(zip(outputs, ppl)))
# print(f"prediction: {outputs[ppl.index(min(ppl))]}")

100%|██████████| 1/1 [00:01<00:00,  1.43s/it]


[47.66353959810042, 15.044857001110119]